<a href="https://colab.research.google.com/github/JaquesZanon/JOB/blob/main/PREVISAO_DEMANDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importa aquivo de texto (.txt)
import pandas as pd
df_original = pd.read_csv('/content/drive/MyDrive/JOB-Martins/DadosPrevisaoDemanda.csv',sep=';', decimal=',', engine='python')

df_original['PrimeiraDataSemana'] = pd.to_datetime(df_original['PrimeiraDataSemana'])
df_original['UltimaDataSemana'] = pd.to_datetime(df_original['UltimaDataSemana'])
df_original = df_original.sort_values(by=['PrimeiraDataSemana'])
df_original.head()


,SiglaUF,DsCategoria,DsSubCategoria,CdGrupoProdutoSimilar,DsGrupoProdutoSimilar,DsMarca,DsDivisaoFornecedor,ANO,PrimeiraDataSemana,UltimaDataSemana,SemanaAno,QtdeVendida,VlTotalBruto,VlTotalLiquido,VlReceitaLiquida,Semanas
0,BA,ACESSORIO ELETRICO,CAIXA DE LUZ,4200765,CXA.LUZ EMB.KRONA 4X2 AM.24X1,Krona,KRONA TUBOS E CONEXOES LTDA.,2020,2020-01-05,2020-01-05,18,1,23.25,23.25,21.10,1
850,BA,SABONETE,SABONETE EM BARRA,205138,SAB.GRANADO 12X90G GLIC.BENJO.,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-01-05,18,3,129.06,124.92,111.72,1
849,BA,SABONETE,SABONETE EM BARRA,205137,SAB.GRANADO 12X90G GLIC.AMEND.,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-01-05,18,3,144.30,139.68,124.92,1
848,BA,SABONETE,SABONETE EM BARRA,203130,SAB.GRANADO 12X90G GLIC.MEL,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-01-05,18,3,144.30,139.68,124.92,1
847,BA,SABONETE,SABONETE EM BARRA,201588,SAB.GRANADO 12X90G GLIC.CALE.,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-02-05,18,4,164.79,154.57,138.24,1


In [4]:
#dat_aux = pd.DataFrame(df_reduzido.groupby(['DsDivisaoFornecedor'])['CdGrupoProdutoSimilar'].unique())
dat_aux = pd.DataFrame(df_original.groupby(['DsDivisaoFornecedor'])['CdGrupoProdutoSimilar'].unique())
dat_aux

,CdGrupoProdutoSimilar
DsDivisaoFornecedor,
3M DO BRASIL LTDA,"[805, 97145, 803, 88, 818, 97147, 406632, 9714..."
3M DO BRASIL LTDA - DIV. FITAS,"[406207, 400433, 229, 406675, 406674, 400434, ..."
3M DO BRASIL LTDA - EPI,"[9000524, 9000527, 9000531]"
3M DO BRASIL LTDA - LIMPEZA,"[1700717, 1704060, 1702339, 200755, 203136, 20..."
3M DO BRASIL LTDA LIMPEZA &&MP,"[1702629, 1701347, 1701348, 1701261, 1701262, ..."
...,...
WHIRLPOOL S.A - L.BCA,"[636, 1041, 191, 2217224, 1793, 1794, 2213230,..."
WORLD COM IND MED VET COSM LIM,"[701869, 701761, 701768, 701767, 701762, 70176..."
WOW INDUSTRIA E COMERCIO LTDA,"[1600715, 1600716, 1600609, 1600455, 1600390, ..."


In [ ]:
# Precisamos instalar algumas Bibliotecas
!pip install utils
!pip install pmdarima # para autoARIMA
!pip install pyEDM # Para Empirical Dynamic Modeling
!pip install croston
!pip install gluonts # PAra DeepAR
!pip install mxnet # PAra Deep AR
!pip install prophet
!pip install tqdm
!pip install sklearn
!pip install --upgrade mxnet~=1.7 gluonts
!pip install darts
!pip install 'u8darts[all]'
!pip install skedm

In [5]:
def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [ ]:
#def percentage_error(actual, predicted):
#   res = np.empty(actual.shape)
#    for j in range(actual.shape[0]):
#        if actual[j] != 0:
#            res[j] = (actual[j] - predicted[j]) / actual[j]
#        else:
#            res[j] = predicted[j] / np.mean(actual)
#    return res
#
#def mean_absolute_percentage_error(y_true, y_pred): 
#    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

In [6]:
from scipy import stats
import numpy as np
import pmdarima as pm
#from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from croston import croston
from prophet import Prophet
from tqdm import tqdm
import time
import pyEDM
from darts import TimeSeries
from darts.metrics import mape
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    RegressionEnsembleModel,
    RegressionModel,
    Theta,
    FFT
)
import skedm as edm

In [7]:
independencia = pd.DataFrame({
  'holiday': 'independencia',
  'ds': pd.to_datetime(['2020-09-13']),
  'lower_window': 0,
  'upper_window': 1,
})
independencia

,holiday,ds,lower_window,upper_window
0,independencia,2020-09-13,0,1


In [32]:
pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0]

0     6368.226259
1     6368.226259
2     6368.226259
3     6368.226259
4     6368.226259
         ...     
59    6368.226259
60    6368.226259
61    6368.226259
62    6368.226259
63    6368.226259
Name: 0, Length: 64, dtype: float64

In [ ]:
par={}
mape_arima={}
par_croston={}
mape_croston={}
mape_prophet={}
mape_EDM = {}
mape_naive = {}
mape_ExponentialSmoothing = {}
mape_Theta = {}
mape_FFT = {}

smape_arima={}
smape_croston={}
smape_prophet={}
smape_EDM = {}
smape_naive = {}
smape_ExponentialSmoothing = {}
smape_Theta = {}
smape_FFT = {}

pred_arima ={}
pred_croston = {}
pred_prophet = {}
pred_EDM = {}
pred_Naive ={}
pred_ExponentialSmoothing={}
pred_Theta = {}
pred_FFT = {}


#for j, i in zip(tqdm (range(0, 7),desc="Processando Fornecedor", ascii=False, ncols=75), tqdm (range(0,len(dat_aux.iloc[j,0])),desc="Processando Id Oriduto", ascii=False, ncols=75)):

for j in tqdm (range(0, 10),desc="Processando Fornecedor", ascii=False, ncols=75):
  
    #try:
   # Selecionando os dados para ajustar os modelos
         filter1 = df_original["DsDivisaoFornecedor"]==dat_aux.index[j]
         #filter1 = df_original["DsDivisaoFornecedor"]=='SAINT GOBAIN DO BRASIL PRODUTOS IND'
         df_0 = df_original[filter1]
         df = df_0.groupby(['Semanas'])[['VlTotalBruto']].sum()
         we = pd.DataFrame(list(range(1,65,1)))
         we.columns=['Semanas']
         we.index=we.Semanas
         df = pd.concat([df, we], axis=1)
         df=df['VlTotalBruto']
         df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=64), columns=['date']).iloc[:,0]
         df = pd.DataFrame(df).fillna(0)

      # Substituindo outliers
         z = np.abs(stats.zscore(df.VlTotalBruto))
         median=df.VlTotalBruto.median()
         df["VlTotalBruto"] = np.where(z>3, median, df["VlTotalBruto"])

      # Sets de treino e teste
         train_len = int(df.shape[0] * 1)
         train_data, test_data = df[:train_len], df[train_len:]
         test_data = train_data

      # Ajustando AutoArima
         stepwise_fit = pm.auto_arima(train_data, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True, d=1, D=1, trace=False,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise
   
      # Armazenando resultados AutoArima
         sentence = [str(dat_aux.index[j])]
         sentence = '-'.join(sentence)
         aux = stepwise_fit.get_params()
         par[sentence]=aux
         forecasts = stepwise_fit.predict_in_sample(test_data.shape[0])
         #mmae = smape(test_data.VlTotalBruto, forecasts)
         mape_arima[sentence]=mean_absolute_percentage_error(list(list(test_data.VlTotalBruto)), list(forecasts))
         pred_arima[sentence]=forecasts
         smape_arima [sentence] = smape(test_data.VlTotalBruto, forecasts)

  
      # Ajustando modelo Croston
         croston_pred = croston.fit_croston(df+0.000001,test_data.shape[0],'original')

      # Armazenando resultados Croston
         par_croston[sentence] = croston_pred
         mape_croston[sentence] = mean_absolute_percentage_error(list(test_data.VlTotalBruto), croston_pred['croston_forecast'])
         smape_croston[sentence] = smape(test_data.VlTotalBruto, pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0])
         pred_croston[sentence] = pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0]


      # O modelo profet só funciona com duas colunas
         y = df.reset_index(drop=False)
         y.columns = ['ds', 'y']
         #y['floor'] = 0
         #y['cap'] = 200000
         train = y.iloc[:train_data.shape[0],:]
         test = y.iloc[train_data.shape[0]:,:]
         test=train

      #Ajustando modelo Prophet
         m = Prophet(weekly_seasonality = True,
                     yearly_seasonality = False,
                     daily_seasonality = False,
                     holidays=independencia)
         m.fit(train,verbose=0)
         future = m.make_future_dataframe(periods=len(test))
         future['ds'] = pd.to_datetime(future['ds']).dt.date
         forecast = m.predict(future)
         one=test['y']
         twi=list(forecast.loc[:test_data.shape[0]-1,'yhat'])
         mape_prophet[sentence] = mean_absolute_percentage_error(list(one),list(twi))# Assimetrico
         smape_prophet[sentence] = smape(one, twi) # simmetrico
         pred_prophet[sentence] = twi

         # O EDM agora, preparando o imput
         y = y[["ds","y"]]
         y.columns = ["Time", "Revenue"]
         y.Time = list(range(1,df.shape[0]+1,1))
         
         aaa=pyEDM.EmbedDimension(dataFrame = y, lib="1 64", pred="1 64", columns="Revenue",target = "Revenue",showPlot=False)
         maxE = aaa.rho.max()
         selectedE=aaa[aaa.rho==maxE]['E']
         intervalo = pyEDM.PredictInterval(dataFrame = y,lib="1 64", pred="1 64", columns="Revenue", E=int(selectedE), showPlot=False)
         maxpred = intervalo.rho.max()
         selectedpred=intervalo[intervalo.rho==maxpred]['Tp']
         simplex_pred = pyEDM.Simplex(dataFrame = y,lib="1 64", pred="1 64", E= 1 if maxE==0 else int(selectedE), columns="Revenue",Tp=int(selectedpred),tau=1)
         #print(pyEDM.ComputeError(simplex_pred['Observations'],simplex_pred['Predictions']))
         simplex_pred = simplex_pred[['Observations','Predictions']].dropna()
         pred_EDM[sentence] = list(simplex_pred['Predictions'])
         mape_EDM[sentence] = mean_absolute_percentage_error(list(simplex_pred['Observations']), list(simplex_pred['Predictions']))
         smape_EDM[sentence] = smape(simplex_pred['Observations'], simplex_pred['Predictions'])

         # PAra Naive, exponential smothing, theta, FFT
         series = TimeSeries.from_dataframe(df.reset_index(), 'date', 'VlTotalBruto')
         train2, val = series.split_after(0.7)
         trains2=val=series
         model1 = NaiveDrift()
         model1.fit(train2+0.00000001)
         pred_val1 = model1.predict(len(val))
         pred_Naive[sentence] = pd.DataFrame(pred_val1.values()).iloc[:,0]
         mape_naive[sentence] = mean_absolute_percentage_error(series.values(),pred_val1.values())
         smape_naive[sentence] = str(smape(series.values(),pred_val1.values()))

         model2 = ExponentialSmoothing()
         model2.fit(train2+0.00000001)
         pred_val2 = model2.predict(len(val))
         pred_ExponentialSmoothing[sentence] = pd.DataFrame(pred_val2.values()).iloc[:,0]
         mape_ExponentialSmoothing[sentence] = mean_absolute_percentage_error(series.values(),pred_val2.values())
         smape_ExponentialSmoothing[sentence] = str(smape(series.values(),pred_val2.values()))

         model3 = Theta()
         model3.fit(train2+0.00000001)
         pred_val3 = model3.predict(len(val))
         pred_Theta[sentence] = pd.DataFrame(pred_val3.values()).iloc[:,0]
         mape_Theta[sentence] = mean_absolute_percentage_error(series.values(),pred_val3.values())
         smape_Theta[sentence] = str(smape(series.values(),pred_val3.values()))

         model4 = FFT(trend='poly',trend_poly_degree = 2)
         model4.fit(train2+0.00000001)
         pred_val4 = model4.predict(len(val))
         pred_FFT[sentence] = pd.DataFrame(pred_val4.values()).iloc[:,0]
         mape_FFT[sentence] = mean_absolute_percentage_error(series.values(),pred_val4.values())
         smape_FFT[sentence] = str(smape(series.values(),pred_val4.values()))

    #except:
      #pass  

resultado_final_mape= pd.concat([pd.DataFrame.from_dict(mape_arima,orient='index'),
           pd.DataFrame.from_dict(mape_croston,orient='index'),
           pd.DataFrame.from_dict(mape_prophet,orient='index'),
           pd.DataFrame.from_dict(mape_EDM,orient='index'),
           pd.DataFrame.from_dict(mape_naive,orient='index'),
           pd.DataFrame.from_dict(mape_ExponentialSmoothing,orient='index'),
           pd.DataFrame.from_dict(mape_Theta,orient='index'),
           pd.DataFrame.from_dict(mape_FFT,orient='index')   
           ], axis=1)
resultado_final_mape.columns=["arima","croston", "prophet","EDM","naive","ExpSmooth","theta","fft"]

resultado_final_smape= pd.concat([pd.DataFrame.from_dict(smape_arima,orient='index'),
           pd.DataFrame.from_dict(smape_croston,orient='index'),
           pd.DataFrame.from_dict(smape_prophet,orient='index'),
           pd.DataFrame.from_dict(smape_EDM,orient='index'),
           pd.DataFrame.from_dict(smape_naive,orient='index'),
           pd.DataFrame.from_dict(smape_ExponentialSmoothing,orient='index'),
           pd.DataFrame.from_dict(smape_Theta,orient='index'),
           pd.DataFrame.from_dict(smape_FFT,orient='index')   
           ], axis=1)
resultado_final_smape.columns=["arima","croston", "prophet","EDM","naive","ExpSmooth","theta","fft"]

predicoes = pd.concat([pd.DataFrame.from_dict(pred_arima,orient='index'),
           pd.DataFrame.from_dict(pred_croston,orient='index'),
           pd.DataFrame.from_dict(pred_prophet,orient='index'),
           pd.DataFrame.from_dict(pred_EDM,orient='index'),
           pd.DataFrame.from_dict(pred_Naive,orient='index'),
           pd.DataFrame.from_dict(pred_ExponentialSmoothing,orient='index'),
           pd.DataFrame.from_dict(pred_Theta,orient='index'),
           pd.DataFrame.from_dict(pred_FFT,orient='index')   
           ], axis=0)
predicoes['modelos']= ["arima"]*10 + ['croston']*10 + ["prophet"]*10 + ["EDM"]*10 + ['naive']*10 + ['ExpSmooth']*10 + ['theta']*10 + ['fft']*10

#resultado_final.to_csv('/content/drive/MyDrive/JOB-Martins/rfesultado_final.csv')
#pd.DataFrame.from_dict(erro_prophet,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/prophet.csv')
#pd.DataFrame.from_dict(erro_croston,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/croston.csv')
#pd.DataFrame.from_dict(erro,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/arima.csv')
#pd.DataFrame.from_dict(erro_EDM,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/EDM.csv')
resultado_final_mape.to_csv('/content/drive/MyDrive/JOB-Martins/mape.csv')
resultado_final_smape.to_csv('/content/drive/MyDrive/JOB-Martins/smape.csv')
predicoes.to_csv('/content/drive/MyDrive/JOB-Martins/predicoes.csv')


print('Cmplete')

In [81]:
resultado_final_mape.to_csv('/content/drive/MyDrive/JOB-Martins/mape.csv')
resultado_final_smape.to_csv('/content/drive/MyDrive/JOB-Martins/smape.csv')
predicoes.to_csv('/content/drive/MyDrive/JOB-Martins/predicoes.csv')

In [87]:

aaa=pyEDM.EmbedDimension(dataFrame = y, lib="1 64", pred="1 64", columns="Revenue",target = "Revenue",showPlot=False)
maxE = aaa.rho.max()
selectedE=aaa[aaa.rho==maxE]['E']
intervalo = pyEDM.PredictInterval(dataFrame = y,lib="1 64", pred="1 64", columns="Revenue", E=int(selectedE), showPlot=False)
maxpred = intervalo.rho.max()
selectedpred=intervalo[intervalo.rho==maxpred]['Tp']
simplex_pred = pyEDM.Simplex(dataFrame = y,lib="1 64", pred="1 64", E= 1 if maxE==0 else int(selectedE), columns="Revenue",Tp=int(selectedpred),tau=1)
#print(pyEDM.ComputeError(simplex_pred['Observations'],simplex_pred['Predictions']))
simplex_pred = simplex_pred[['Observations','Predictions']].dropna()
pred_EDM[sentence] = list(simplex_pred['Predictions'])
mape_EDM[sentence] = mean_absolute_percentage_error(list(simplex_pred['Observations']), list(simplex_pred['Predictions']))


In [ ]:
resultado_final.to_csv('/content/drive/MyDrive/JOB-Martins/rfesultado_final.csv')

In [ ]:
resultado_final.to_excel('/content/drive/MyDrive/JOB-Martins/rfesultado_final.xlsx')

In [54]:
simplex_pred['Observations'].dropna()

0       542.840
1      2277.860
2      6461.830
3      3404.370
4      1445.680
        ...    
59     4024.195
60    12199.240
61     9321.210
62     9359.130
63     4168.910
Name: Observations, Length: 64, dtype: float64

In [ ]:
df_original.columns

Index(['SiglaUF', 'DsCategoria', 'DsSubCategoria', 'CdGrupoProdutoSimilar',
       'DsGrupoProdutoSimilar', 'DsMarca', 'DsDivisaoFornecedor', 'ANO',
       'PrimeiraDataSemana', 'UltimaDataSemana', 'SemanaAno', 'QtdeVendida',
       'VlTotalBruto', 'VlTotalLiquido', 'VlReceitaLiquida', 'Semanas'],
      dtype='object')

In [ ]:
resultado = {}
df_2 = df
for j in range(0, 441):
   filter1 = df_original["DsDivisaoFornecedor"]==dat_aux.index[440]
   df_0 = df_original[filter1]
   df = df_0.groupby(['Semanas'])[['QtdeVendida']].sum()
   we = pd.DataFrame(list(range(1,65,1)))
   we.columns=['Semanas']
   we.index=we.Semanas
   df = pd.concat([df, we], axis=1)
   df=df['QtdeVendida']
   df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=64), columns=['date']).iloc[:,0]
   df = pd.DataFrame(df).fillna(0)
   df.columns = [dat_aux.index[j]]
   df_2[dat_aux.index[j]]=df
   #resultado[dat_aux.index[j]] = df.reset_index()


In [ ]:
df

,QtdeVendida
Semanas,
1,246
2,750
3,1300
4,872
5,974
...,...
60,2962
61,1388
62,1383


In [ ]:

df_2.to_excel('/content/drive/MyDrive/JOB-Martins/quantidade_vendida_Series.xlsx')